In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
data = load_iris()
data.data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [ ]:
X = data.data
y = data.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train =SC.fit_transform(X_train)
X_test = SC.fit_transform(X_test)

In [ ]:
ohe = OneHotEncoder(sparse_output=False)
SC = StandardScaler()

In [ ]:
y_train = ohe.fit_transform(y_train.reshape(-1, 1))
y_test = ohe.transform(y_test.reshape(-1, 1))

In [ ]:
y_train[:3,:]

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [ ]:
X_train_tensor = torch.tensor(X_train,dtype=torch.float32)
X_test_tensor = torch.tensor(X_test,dtype=torch.float32)
y_train_tensor = torch.tensor(y_train,dtype=torch.float32)
y_test_tensor = torch.tensor(y_test,dtype=torch.float32)

In [ ]:
class iris_class(nn.Module):
  def __init__(self):
    super(iris_class,self).__init__()
    self.fc1= nn.Linear(4,10)
    self.fc2= nn.Linear(10,3)

  def forward(self,inp):
    inp = F.relu(self.fc1(inp))
    inp = self.fc2(inp)
    return inp

In [ ]:
model = iris_class()

In [ ]:
criterion =nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [ ]:
num_epochs = 80
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs =model(X_train_tensor)
  loss = criterion(outputs,y_train_tensor)
  loss.backward()
  optimizer.step()

  if (epoch+1) % 5 ==0:
    print(f'epoch [{epoch+1}/{num_epochs}], loss: {loss.item():.4f}')

epoch [5/80], loss: 0.0645
epoch [10/80], loss: 0.0616
epoch [15/80], loss: 0.0592
epoch [20/80], loss: 0.0570
epoch [25/80], loss: 0.0551
epoch [30/80], loss: 0.0534
epoch [35/80], loss: 0.0518
epoch [40/80], loss: 0.0504
epoch [45/80], loss: 0.0492
epoch [50/80], loss: 0.0480
epoch [55/80], loss: 0.0470
epoch [60/80], loss: 0.0460
epoch [65/80], loss: 0.0451
epoch [70/80], loss: 0.0443
epoch [75/80], loss: 0.0436
epoch [80/80], loss: 0.0429


In [ ]:
model.eval()
with torch.no_grad():
  test_outputs = model(X_test_tensor)
  predictions = torch.sigmoid(test_outputs)
  predicted_classes = predictions.argmax(dim=1)

In [ ]:
predicted_classes

tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 2,
        2, 2, 2, 2, 0, 0])

In [ ]:
actual_classes = y_test_tensor.argmax(dim=1)
accuracy = (predicted_classes == actual_classes).float().mean()
accuracy

tensor(0.9667)

In [ ]:
accuracy_score = accuracy.item()
accuracy_score

0.9666666388511658

In [ ]:
type(predicted_classes)

torch.Tensor

In [ ]:
pred_numpy = predicted_classes.numpy()

In [ ]:
print(type(y_test))
print(type(pred_numpy))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
from sklearn.metrics import f1_score,precision_score,classification_report

In [ ]:
y_test_labels = np.argmax(y_test, axis=1)

In [ ]:
y_test_labels[:3]

array([1, 0, 2])

In [ ]:
pred_numpy[:3]

array([1, 0, 2])

In [ ]:
f1_report = f1_score(pred_numpy,y_test_labels,average='macro')

In [ ]:
f1_report

0.9665831244778612

In [ ]:
precision = precision_score(y_test_labels,pred_numpy,average='micro')
precision

0.9666666666666667

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_labels,pred_numpy)

In [ ]:
matrix

array([[10,  0,  0],
       [ 0,  9,  0],
       [ 0,  1, 10]])

In [ ]:
inference_input =np.array([[ 2.07023667, -1.16327782,  1.66633891,  1.30025404]])

In [ ]:
inference_tensor = torch.tensor(inference_input,dtype=torch.float32)

In [ ]:
X_train_tensor.dtype

torch.float32

In [ ]:
model.eval()
with torch.no_grad():
  inference_output = model(inference_tensor)
  probabilities = F.softmax(inference_output, dim=1)
  predicted_class = torch.argmax(probabilities, dim=1).item()

In [ ]:
predicted_class

2